In [ ]:
# This file was succesfully tested on Julia 1.3.1
# Generates results for the Short-Run decomposition in the Frequency Domain
# replacing the GDP deflator based inflation measure by the CORE based (starts in 1957Q2).
include("../toolboxes/toolbox.jl");

In [ ]:
gbseed = 1234567890 # Seed to be used in the Gibbs sampler
alpha  = 68         # We want 68% confidence bands
lags   = 2          # Number of lags in the VAR
gibbs_ = true       # false: if false, uses results stored in the filegibbs file 
T0     = 1957.25    # Initial date of sample (1960Q1 -> 1960, 1960Q2-> 1960.25, 1960Q3-> 1960.50, 1960Q4-> 1960.75)
T1     = 2017.75    # Final date of sample
Ndraws = 50000      # Total number of draws in Gibbs sampling
Nburn  = 49000      # Burn in iterations
maxhor = 200        # Maximal horizon when testing for LR effects
nrep   = 40         # Number of periods 
test   = 1          # Testing period for the IRF
save_  = true       # set to true to save results in a matlab file

#
# Specifying the type of analysis
#   
#
# In case of Frequency domain:
# wmin -> lowest frequency to consider  : BC analysis, wmin=2π/32, LR analysis, wmin=1e-5
# wmax -> highest frequency to consider : BC analysis, wmax=2π/6,  LR analysis, wmax=2π/80  
wmin    = 2*π/32       
wmax    = 2*π/6        

Idv     = [5,6,8]    # index of Variable (in the estimated VAR) for which we maximize the variance contribution
                       # Here: 1->Y, 2->I, 3->C, 4->h ... (see below list of variables in the VAR)
                       # assume that instead of looking at Y in isolation, you want to maximize over Y and u at the same time
                       # then Idv would look like Idv=[[1,5],2,3,4,5,6,7,8,9,10]
ncase   = length(Idv)

filegibbs="../results_var/core_var_1955_2017_fd_sr.mat" # File containing Gibbs results (if already ran) 
fileres = "../results_var/core_var_1955_2017_fd_sr.mat" # Result file

# Read and prepare data (stored in a matlab data file)
#************************************
# First read the data:
# Column Variable
# 1      Quarters
# 2      Y   : log real GDP
# 3      I   : log real Investment
# 4      C   : log real Consumption
# 5      h   : log total Hours
# 6      u   : Unemployment Rate
# 7      Wh/Y: log Labor Share
# 8      FFR : Federal Funds Rate
# 9      ΔP  : Inflation Rate
# 10     Y/h : Labor productivity
# 11     TFP : Total Factor Productivity
#************************************
data   = matread("../data/data_1955_2017.mat")
data   = data["X"]
time   = data[:,1]
data   = data[T0.<=time.<=T1,2:11] 


# Replaces the GDP deflator based quarterly
datinf = matread("../data/inflation_data.mat")
Tinf   = datinf["time"]
dcore  = datinf["inf_core"][T0.<=Tinf.<=T1]
data[:,8]=copy(dcore)

# Y, I, C, h, u, Wh/Y, FFR, Inflation, Y/h, TFP                                 
# 1  2  3  4  5  6     7    8          9    10 
########################################################################################################
#
# Compute the posterior distribution of the VAR
#
########################################################################################################
# Priors
AR     = [] # used to build priors (if empty, all variables have a RW prior)
priors = minnesota_priors(data,AR,lags) # builds the prior object
resvar = varest(data,lags)              # estimates the VAR

# Gibbs Sampling
if gibbs_
    gibbs = gibbs_var(resvar,priors,Ndraws,Nburn;seed=gbseed)
    VOL   = gibbs.vol;
    COEFS = gibbs.coefs;
else
    tmp   = matread(filegibbs)
    VOL   = tmp["VOL"]
    COEFS = tmp["COEFS"]    
end

# Build a selection matrix of variables 
# (useful for the state space representation of the VAR)
# y i c h u sw R pi y/h TFP 
# 1 2 3 4 5 6  7 8  9   10
nvar    = resvar.nvar
Ident   = eye(nvar)
MYtmp   = [eye(nvar) zeros(nvar,nvar*(lags-1))]
MY      = MYtmp[1:10,:]
ny      = size(MY,1)

########################################################################################################
#
# Build IRFs, Variance decomposition and Forecast errors decomposition
#
########################################################################################################

# Storage:
# 1st index: case under consideration (contained in Idv)
# 2nd index: draws 
# 3rd index: variable or variable*horizon (for IRFs and FEV)
Qmat    = zeros(ncase,Ndraws-Nburn,nvar)        # Impulse vector
IRFsr   = zeros(ncase,Ndraws-Nburn,ny*nrep)     # contains IRFs
SVTsr   = zeros(ncase,Ndraws-Nburn,ny)          # Short unr contribution (6-32 quarters)
SVTlr   = zeros(ncase,Ndraws-Nburn,ny)          # Long run contribution (80-inf quarters)
VD      = zeros(ncase,Ndraws-Nburn,ny*maxhor)   # standard FEV

p = Progress(Ndraws-Nburn,1,"Progress ... ",80)
for i in 1:Ndraws-Nburn
    update!(p,i)
    # 1) Build the state space representation of the VAR for draw i
    tmp   = VOL[i,:];
    S     = Array(chol(Hermitian(reshape(tmp,nvar,nvar))))'
    tmp   = reshape(COEFS[i,:]',nvar*lags+1,nvar)'
    DYN   = tmp[:,1:nvar*lags]
    MX    = [DYN;eye(nvar*(lags-1)) zeros(nvar*(lags-1),nvar)]
    ME    = [S;zeros(nvar*(lags-1),nvar)]
    modvar= state_space_model(MX,MY,ME,eye(nvar))
    # 2) Compute IRFs and cumulative squared IRFs from Choleski decomposition
    Itmp1 = zeros(nrep*ny,nvar)    # IRF
    Itmp2 = zeros(ny,nvar)         # IRF at test horizon (to guarantee sign)
    Itmp3 = zeros(maxhor*ny,nvar)  # cumulative squared IRF 
    for k in 1:nvar
        tmp        = comp_irf(modvar,k,maxhor)
        ttmp       = tmp.y[1:nrep,:]
        Itmp1[:,k] = ttmp[:]
        Itmp2[:,k] = tmp.y[test,:]'
        vtmp       = tmp.y[1:maxhor,:]
        Itmp3[:,k] = vtmp[:]
    end
    # 3) Computes the MBC shock, then the IRFs, FEV and variance decomposition to this shock.
    QQ=zeros(nvar,ncase)    
    for ic in 1:ncase
        idx             = Idv[ic]
        nidx            = length(idx)
        weight          = ones(1,nidx)/nidx  # weight to assign to each variable when trying to maximize other several variables.        
        Q               = funcfdq(modvar,S,wmin,wmax,idx,weight)
        sgn             = sign((Itmp2[idx[1],:]'*Q)[1])
        Qmat[ic,i,:]    = sgn*Q
        IRFsr[ic,i,:]   = sgn*(Itmp1*Q)
        y1,y2           = share_trans(modvar,S,Q)
        SVTsr[ic,i,:]   = y1
        SVTlr[ic,i,:]   = y2
        VDtmp           = vdec(Itmp3,Q,maxhor)
        VD[ic,i,:]      = VDtmp[:]
    end
end
# Generate IRF, FEV and Variance decomposition average, median and confidence bands
airf   = zeros(ncase,nrep,ny)
mirf   = zeros(ncase,nrep,ny)
lirf   = zeros(ncase,nrep,ny)
sirf   = zeros(ncase,nrep,ny)
afev   = zeros(ncase,maxhor,ny)
mfev   = zeros(ncase,maxhor,ny)
lfev   = zeros(ncase,maxhor,ny)
sfev   = zeros(ncase,maxhor,ny)
asvtsr = zeros(ncase,ny)
msvtsr = zeros(ncase,ny)
lsvtsr = zeros(ncase,ny)
ssvtsr = zeros(ncase,ny)
asvtlr = zeros(ncase,ny)
msvtlr = zeros(ncase,ny)
lsvtlr = zeros(ncase,ny)
ssvtlr = zeros(ncase,ny)
for ic in 1:ncase
    airf[ic,:,:] = reshape(mean(IRFsr[ic,:,:],dims=1),nrep,ny)
    mirf[ic,:,:] = reshape(median(IRFsr[ic,:,:],dims=1),nrep,ny)
    pcl          = [percentile(IRFsr[ic,:,i],(100-alpha)/2) for i=1:nrep*ny]
    pcs          = [percentile(IRFsr[ic,:,i],100-(100-alpha)/2) for i=1:nrep*ny]
    lirf[ic,:,:] = reshape(pcl,nrep,ny)
    sirf[ic,:,:] = reshape(pcs,nrep,ny)

    afev[ic,:,:] = reshape(mean(VD[ic,:,:],dims=1),maxhor,ny)
    mfev[ic,:,:] = reshape(median(VD[ic,:,:],dims=1),maxhor,ny)
    pcl          = [percentile(VD[ic,:,i],(100-alpha)/2) for i=1:maxhor*ny]
    pcs          = [percentile(VD[ic,:,i],100-(100-alpha)/2) for i=1:maxhor*ny]
    lfev[ic,:,:] = reshape(pcl,maxhor,ny)
    sfev[ic,:,:] = reshape(pcs,maxhor,ny)
    
    
    asvtsr[ic,:] = mean(SVTsr[ic,:,:],dims=1)
    msvtsr[ic,:] = median(SVTsr[ic,:,:],dims=1)
    lsvtsr[ic,:] = [percentile(SVTsr[ic,:,i],(100-alpha)/2) for i=1:ny]
    ssvtsr[ic,:] = [percentile(SVTsr[ic,:,i],100-(100-alpha)/2) for i=1:ny]

    asvtlr[ic,:] = mean(SVTlr[ic,:,:],dims=1)
    msvtlr[ic,:] = median(SVTlr[ic,:,:],dims=1)
    lsvtlr[ic,:] = [percentile(SVTlr[ic,:,i],(100-alpha)/2) for i=1:ny]
    ssvtlr[ic,:] = [percentile(SVTlr[ic,:,i],100-(100-alpha)/2) for i=1:ny]
end


if save_
    fres = matopen(fileres, "w")
    write(fres,"airf",airf)
    write(fres,"mirf",mirf)
    write(fres,"lirf",lirf)
    write(fres,"sirf",sirf)
    write(fres,"afev",afev)
    write(fres,"mfev",mfev)
    write(fres,"lfev",lfev)
    write(fres,"sfev",sfev)
    write(fres,"asvtsr",asvtsr)
    write(fres,"msvtsr",msvtsr)
    write(fres,"lsvtsr",lsvtsr)
    write(fres,"ssvtsr",ssvtsr)
    write(fres,"asvtlr",asvtlr)
    write(fres,"msvtlr",msvtlr)
    write(fres,"lsvtlr",lsvtlr)
    write(fres,"ssvtlr",ssvtlr)
    write(fres,"Qmat",Qmat)
    write(fres,"VOL",VOL)
    write(fres,"COEFS",COEFS)
    close(fres)
end